# Solving the PAP in a finite A and finite $\chi$ instance

Before solving the CC-PAP, we try to solve the PAP in a discrete instance.

In [1]:
# imports

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt

We will work on the following example
- $A = \{1,2,4,6\}$
- $\mathcal{X} = \{1,2,3,4,5\}$
- $\forall x \in \mathcal{X},\, \forall a \in A, \quad F(x|a) = \mathbb{P}^a[X \leq x] = (\frac{x}{5})^a$

and we will take as utility functions

$$
\begin{cases}
U_A(s,a) = u(s) - a^2\\
U_P(x,s) = x - s 
\end{cases}
$$

In [2]:
set_a = [1,2,4,6]
set_x = [1,2,3,4,5]
extended_set_x = np.arange(max(set_x)+1)
max_set_x = max(set_x)

## $u = \ln$, original variables

First, let us define $\texttt{prob\_vect(a)}$, which returns the ndarray
$$
[p_1(a),\dots,p_5(a)]
$$

In [3]:
def prob_vect(a):
    cdf_vect = (extended_set_x/max_set_x)**a
    return cdf_vect[1:]-cdf_vect[:-1]

Then, we define $\texttt{get\_expected\_ua\_function(a)}$, which returns
$$
\left (\sum_{i=1}^5 p_i(a)\ln(w_i) \right ) - a^2
$$

And $\texttt{get\_expected\_up\_function(a)}$, which returns
$$
\sum_{i=1}^5 p_i(a)w_i
$$

Note that the latter function is not really the utility function of the principal, but the objective function of $(P_a)$ (its equivalent for the PAP, _cf_ paper).

In [4]:
def get_expected_ua_function(a):
    return lambda w:np.dot(prob_vect(a),np.log(w)) - a**2

def get_expected_up_function(a):
    return lambda w:np.dot(prob_vect(a),w)

In [5]:
# bounds and parameters of the problem

bounds = sp.optimize.Bounds(lb = len(set_x)*[1e-9], ub=len(set_x)*[np.inf])


In [6]:
def get_participation_const(i:int,R0=0,ua_fun=get_expected_ua_function):
    return sp.optimize.NonlinearConstraint(
        fun = ua_fun(set_a[i]),
        lb = [R0],
        ub = [np.inf]
    )

In [7]:
def get_agent_rationality_const_list(i:int,ua_fun=get_expected_ua_function):

    const_list = []
    a = set_a[i]


    for j in range(len(set_a)):
        if i != j:
            
            const_list.append(sp.optimize.NonlinearConstraint(
                fun = lambda w,j=j:ua_fun(a)(w)-ua_fun(set_a[j])(w),
                lb = [0],
                ub = [np.inf]
            ))
    
    return const_list

With everything defined, let us solve the subproblem $(P_4)$ (its equivalent for the PAP). If it works, we will try to solve all the $(P_a)$ s, and compute the original solution of the PAP. 

In [8]:
init = np.array([5,5,5,5,5]) + np.random.randn(5)
init = np.maximum(init,1e-3)

const_list = get_agent_rationality_const_list(2)+[get_participation_const(2)]

res = sp.optimize.minimize(fun = get_expected_up_function(2), x0 = init,method="trust-constr",
                           constraints=const_list,
                           bounds=bounds)

print(res)

/tmp/ipykernel_10564/409815884.py:2: RuntimeWarning: invalid value encountered in log
  return lambda w:np.dot(prob_vect(a),np.log(w)) - a**2


ValueError: array must not contain infs or NaNs

It seems that during the solving, negative values for $\texttt{x}$ occur, which crashes with the logarithmic utility function of the agent. Let us try a second solving, with a truncated objective function

In [9]:
def get_expected_ua_function_bis(a):
    return lambda w:np.dot(prob_vect(a),np.log(np.maximum(w,1e-9))) - a**2

In [10]:
init = np.array([5,5,5,5,5]) + np.random.randn(5)

const_list = get_agent_rationality_const_list(2,ua_fun=get_expected_ua_function_bis)
const_list.append(get_participation_const(2,ua_fun=get_expected_ua_function_bis))

res = sp.optimize.minimize(fun = get_expected_up_function(2), 
                           x0 = init,method="trust-constr",
                           constraints=const_list,bounds=bounds)

print(res)

/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:552: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


           message: The maximum number of function evaluations is exceeded.
           success: False
            status: 0
               fun: 69884687751.27507
                 x: [-5.009e-03 -9.549e-04 -1.336e-05  2.050e+01  1.941e+11]
               nit: 1000
              nfev: 6000
              njev: 1000
              nhev: 0
          cg_niter: 1741
      cg_stop_cond: 4
              grad: [-0.000e+00 -0.000e+00 -0.000e+00  0.000e+00  3.600e-01]
   lagrangian_grad: [-3.681e-07 -3.263e-05 -3.309e-05  1.468e-04  3.597e-01]
            constr: [array([ 5.137e+00]), array([-1.237e+00]), array([ 1.464e+01]), array([-2.494e+00]), array([-5.009e-03, -9.549e-04, -1.336e-05,  2.050e+01,
                            1.941e+11])]
               jac: [array([[-0.000e+00, -0.000e+00, -0.000e+00,
                             3.902e-03,  2.011e-12]]), array([[-0.000e+00, -0.000e+00, -0.000e+00,
                            -2.907e-09,  1.187e-12]]), array([[-0.000e+00, -0.000e+00, -0.000e+00,

This solving does not succeed, and returns negative values (which somewhat makes sense, as we truncated part of the logarithmic barrier). Overall, this solving method does not work. Let us try the change of variables of Grossman and Hart.

## $u=\ln$, utility variables

The change of variables suggested by Grossman and Hart is the following
$$
\forall i \in [\![1,5]\!],\, v_i = \ln (w_i) \Leftrightarrow w_i = \exp(v_i)
$$

- $\texttt{get\_expected\_ua\_function\_cv(a)}$ should return
$$
\left (\sum_{i=1}^5 p_i(a)v_i \right ) - a^2
$$

As it is affine in $v$, we will not re-implement it

- $\texttt{get\_expected\_up\_function\_cv(a)}$ returns
$$
\left (\sum_{i=1}^5 p_i(a)e^{v_i} \right ) - a^2
$$

As before, the latter function is not really the utility function of the principal, but the objective function of $(P_a)$ (its equivalent for the PAP, _cf_ paper).

In [11]:
def get_expected_up_function_cv(a):
    return lambda v:np.dot(prob_vect(a),np.exp(v))

In [12]:
# bounds and parameters of the problem

R0 = 0
bounds = sp.optimize.Bounds(lb = len(set_x)*[-np.inf], ub=len(set_x)*[np.inf])


Now, let us implement the participation constraint

In [13]:
def get_participation_const_cv(i,R0):
    return sp.optimize.LinearConstraint(
        A = prob_vect(set_a[i]),
        lb = [R0 + set_a[i]**2]
    )

And the agent rationality constraints

In [14]:
def get_agent_rationality_const_list_cv(i:int):

    other_a_list = []

    for j in range(len(set_a)):
        if j != i:
            other_a_list.append(j)

    matrix = np.stack([prob_vect(set_a[i]) - prob_vect(set_a[i_a]) for i_a in other_a_list])

    return sp.optimize.LinearConstraint(
                A = matrix,
                lb = [set_a[i]**2 - set_a[i_a]**2 for i_a in other_a_list]
    )

We have defined everything to solve our problem. Let us use a solver 

In [ ]:
init = np.array([5,5,5,5,5]) + np.random.randn(5)
init = np.maximum(init,1e-3)
print(init)

const_list = [get_agent_rationality_const_list_cv(2),get_participation_const_cv(2,R0=0)]

res = sp.optimize.minimize(fun = get_expected_up_function_cv(set_a[2]), x0 = init,method="trust-constr",
                           constraints=const_list, bounds=bounds)

print(res)

[5.78767422 5.68332153 5.90744422 4.1494163  3.6326688 ]


/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: 14619851.7691663
                 x: [-2.954e+02  1.632e+01  1.646e+01  1.650e+01  1.651e+01]
               nit: 244
              nfev: 2100
              njev: 350
              nhev: 0
          cg_niter: 586
      cg_stop_cond: 4
              grad: [-0.000e+00  2.924e+05  1.462e+06  4.094e+06  8.772e+06]
   lagrangian_grad: [-2.127e-03  4.743e-03 -4.625e-03  1.967e-03 -3.054e-04]
            constr: [array([ 6.191e+01,  1.200e+01, -4.874e-01]), array([ 1.600e+01]), array([ 5.788e+00,  5.683e+00,  5.907e+00,  4.149e+00,
                            3.633e+00])]
               jac: [array([[-1.984e-01, -1.760e-01, ...,  8.000e-02,
                             3.904e-01],
                           [-3.840e-02, -9.600e-02, ..., -5.551e-17,
                             2.304e-01],
                           [ 1.536e-03,  1.997e-02, ...,  6.451e-02,
        

It seems that the solving works ! To be sure of that, let us compute the average utility function of the agent in $\texttt{res.x},\texttt{a}$ for any $\texttt{a} \in A$. The objective is to check that the argmax constraint is satisfied

In [16]:
for i in range(len(set_a)):
    print(f"a = {set_a[i]}")
    print(f"Utility = {get_expected_ua_function(set_a[i])(np.exp(res.x))}")
    print()

a = 1
Utility = -46.91306532060885

a = 2
Utility = -2.758131500968375e-10

a = 4
Utility = 2.0953905277565354e-11

a = 6
Utility = -19.512570679656786



Apart from numerical errors, the argmax constraint is satisfied, and the participation constraint is also satisfied. Now, we can solve the $|A| = 4$ subproblems which arise. 

In [17]:
def solve_pap(i:int,loc_init=np.array([5,5,5,5,5]),loc_bounds=bounds,loc_R0=0,
              loc_obj_func=get_expected_up_function_cv):

    init = loc_init + np.random.randn(5)
    const_list = [get_agent_rationality_const_list_cv(i),get_participation_const_cv(i,R0=loc_R0)]

    res = sp.optimize.minimize(fun = loc_obj_func(set_a[i]), x0 = init,method="trust-constr",
                            constraints=const_list,options={"factorization_method":"SVDFactorization"},
                            bounds=loc_bounds)

    return res

In [18]:
res_list = [solve_pap(i) for i in range(len(set_a))]

for res in res_list:
    print(res.success)

/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can b

True
True
True
True


It seems that all subproblems have been successfully solved : we just have to keep the best $a$ value for the principal to solve the PAP of parameter $R_0$

In [19]:
i_max = -1
best_val = -np.inf

for i in range(len(set_a)):
    if res_list[i].success and (np.dot(set_x,prob_vect(set_a[i])) - res_list[i].fun > best_val):
        i_max = i
        best_val = np.dot(set_x,prob_vect(set_a[i])) - res_list[i].fun

print(f"The result of the PAP is reached for a = {set_a[i_max]}")
print(f"And the optimal wage is {np.exp(res.x)}")

The result of the PAP is reached for a = 1
And the optimal wage is [0.00000000e+00 7.87086866e+15 9.32075986e+15 9.79358033e+15
 9.99986823e+15]


We have successfully solved the PAP of parameter $R_0 = 0$. However, the wages in original variables are huge, because of the exponential variable change. To avoid numerical explosion, we will choose less explosive functions in our next examples.

## $u = -\frac{1}{\sqrt{\cdot}}$, utility variables

We will take in that section
$$
u \colon \left | \begin{matrix}
\mathbb{R}_+^* \to \mathbb{R}_-^*\\
w \mapsto -\frac{1}{\sqrt{w}}
\end{matrix} \right.
$$ 

$u$ is also increasing and concave : we therefore check assumption (C3)

The inverse function of $u$ is

$$
\phi \colon \left | \begin{matrix}
\mathbb{R}_-^* \to \mathbb{R}_+^*\\
v \mapsto \frac{1}{v^2}
\end{matrix} \right.
$$

which is indeed convex and increasing on $\mathbb{R}_-^*$

We will directly use the change of variables in our resolution. Thus, the agent utility function stays the same, we just have to redefine the principal utility function.

In [20]:
def get_expected_up_function_cv_2(a):
    return lambda v:np.dot(prob_vect(a),np.pow(v,-2))

In [21]:
# bounds and parameters of the problem

bounds = sp.optimize.Bounds(lb=len(set_x)*[-np.inf],ub = len(set_x)*[-1e-9])


In [22]:
res = solve_pap(2,loc_init=np.ones(5)*(-5),
          loc_bounds=bounds,loc_R0=-30,loc_obj_func=get_expected_up_function_cv_2)
print(res)

           message: `gtol` termination condition is satisfied.
           success: True
            status: 1
               fun: 0.005476342156597801
                 x: [-3.230e+02 -1.435e+01 -1.369e+01 -1.351e+01 -1.344e+01]
               nit: 84
              nfev: 468
              njev: 78
              nhev: 0
          cg_niter: 215
      cg_stop_cond: 4
              grad: [ 9.496e-11  1.623e-05  8.113e-05  2.272e-04  4.868e-04]
   lagrangian_grad: [-4.724e-10  1.992e-10 -1.014e-09  1.250e-09 -4.208e-10]
            constr: [array([ 6.160e+01,  1.200e+01, -5.139e-01]), array([-1.400e+01]), array([-3.230e+02, -1.435e+01, -1.369e+01, -1.351e+01,
                           -1.344e+01])]
               jac: [array([[-1.984e-01, -1.760e-01, ...,  8.000e-02,
                             3.904e-01],
                           [-3.840e-02, -9.600e-02, ..., -5.551e-17,
                             2.304e-01],
                           [ 1.536e-03,  1.997e-02, ...,  6.451e-02,
       

`scipy.optimize.minimize` seems to have worked. As before, let us check both constraints, _i.e._ that the agent utility is maximal for $a = 4$, and higher or equal to -30. 

In [23]:
def get_expected_ua_orig_variables(a):
    return lambda w:np.dot(prob_vect(a),-np.pow(w,-0.5))-a**2

for i in range(len(set_a)):
    print(f"a = {set_a[i]}")
    print(f"Utility = {get_expected_ua_orig_variables(set_a[i])(np.pow(res.x,-2))}")
    print()

a = 1
Utility = -76.60138374067489

a = 2
Utility = -30.00021288457413

a = 4
Utility = -29.999984958866666

a = 6
Utility = -49.48608352010862



This solution is indeed feasible, which reassures us on the well-working of `solve_pap` and the new objective function.

In [24]:
res_list = [solve_pap(i,loc_init=np.ones(5)*(-5),loc_bounds=bounds,loc_R0=-30,
          loc_obj_func=get_expected_up_function_cv_2) for i in range(len(set_a))]

for res in res_list:
    print(res.success)

i_max = -1
best_val = -np.inf

for i in range(len(set_a)):
    if res_list[i].success and (np.dot(set_x,prob_vect(set_a[i])) - res_list[i].fun > best_val):
        i_max = i
        best_val = np.dot(set_x,prob_vect(set_a[i])) - res_list[i].fun

print(f"The result of the PAP is reached for a = {set_a[i_max]}")
print(f"And the optimal wage is {np.exp(res.x)}")

/home/noe/.virtualenvs/stagecmm/lib/python3.10/site-packages/scipy/optimize/_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


True
True
True
False
The result of the PAP is reached for a = 4
And the optimal wage is [ 0.          1.01374175  1.18601905  2.4783786  23.88388337]


In this instance, the solving algorithm works and does not return huge utility values. Thus, we will use it to solve the ethical CC-PAP